In [1]:


import argparse
from datetime import date
import http.client as hc
from bs4 import BeautifulSoup

def init_argparser() -> argparse.ArgumentParser:
    argparser = argparse.ArgumentParser(description="Scrappy KU site for class info")
    argparser.add_argument("department", help = "department code (EECS, CLSX, ...) to look search for")
    argparser.add_argument("output", help = "file to output results to")
    return argparser

semesters = ["spring", "summer", "fall"]

def searchTermEncoder(year, semester):
    year = year % 100 # only last 2 digist
    if (semester == "spring"):
        return "4" + str(year) + "2"
    elif (semester == "summer"):
        return "4" + str(year) + "6"
    else:
        return "4" + str(year) + "9"

# https: // classes.ku.edu/Classes/CourseSearch.action?classesSearchText = eecs & searchCareer = UndergraduateGraduate & searchTerm = 4219

def traverseClasses(department : str):
    print("dull")
    

# if __name__ == "__main__":
#     argparser = init_argparser()
#     args = argparser.parse_args()
#     dept : str = args.department
#     output : str = args.output



In [11]:

def connect_url(department: str, searchTerm: str) -> str:
    url = f"https://classes.ku.edu/Classes/CourseSearch.action?classesSearchText={department}&searchCareer=UndergraduateGraduate&searchTerm={searchTerm}"
    # open initial connection
    conn = hc.HTTPSConnection("classes.ku.edu")
    conn.request("GET", url)
    resp = conn.getresponse()
    if (resp.status != 200):
        raise ConnectionError("GET failed")
    return resp.read()



In [7]:
htmlText = connect_url("eecs", "4209")

In [23]:
def parse_row(row):
    # return tuple of (className, miniDesc, description)
    descRow = row.find_next_sibling("tr")
    className = row.find("h3").get_text()
    miniDesc = row.find("td").get_text()
    description = descRow.find("td").get_text()
    return (className, miniDesc, description)

def parse_html(text: str):
    soup = BeautifulSoup(text, "html.parser")
    startBlocks = soup.find_all("h3")
    topRows = []
    for blk in startBlocks:
        topRows.append(blk.parent.parent)
    outStuff = []
    for row in topRows:
        outStuff.append(parse_row(row))
    return outStuff

In [26]:
import csv
with open("./out.csv", "w") as out_write:
    csvw = csv.writer(out_write, quoting=csv.QUOTE_ALL)
    for val in parse_html(htmlText):
        csvw.writerow(val)